## 1. Importação

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from typing import List, Dict, Tuple

path_to_dir = '../../raw/Presencial2025_dados_raw/'

df_presencial_cursos = pd.read_csv(f'{path_to_dir}dados_presencial_cursos.csv')
df_presencial_disciplinas = pd.read_csv(f'{path_to_dir}dados_presencial_disciplinas.csv')
df_presencial_pesquisa = pd.read_csv(f'{path_to_dir}/dados_presencial_pesquisa.csv')
df_info_questionario = pd.read_csv(f'{path_to_dir}info_questionario_presencial.csv')
df_info_perguntas = pd.read_csv(f'{path_to_dir}info_perguntas_presencial.csv')
df_info_dimensoes = pd.read_csv(f'{path_to_dir}info_dimensoes_presencial.csv')

## 2. Entendendo cada DataFrame

In [2]:
def dist_variavel(df,column,savefig=True,path_to_dir='./temp_images_presencial/',title='teste'):
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=f'{column}', order=df[f'{column}'].value_counts().index)
    plt.xticks(rotation=45)
    plt.title(f'Distribuição de {column}')
    plt.ylabel('Contagem')
    plt.xlabel(f'{column}')
    plt.tight_layout()
    if savefig:
        plt.savefig(f'{path_to_dir}{title}')
    plt.close()

### 2.1 `df_presencial_cursos`

* `COD_CURSO`
* `CURSO` : Necessita de split (CURSO,PERIODO,LOCALIDADE)
* `SETOR_CURSO`

In [3]:
df_presencial_cursos['TEMP'] = df_presencial_cursos['CURSO'].str.split('-').apply(lambda x: [item.strip() for item in x])

In [4]:
df_presencial_cursos['MODALIDADE'] = df_presencial_cursos['TEMP'].str[1]
df_presencial_cursos['LOCAL'] = df_presencial_cursos['TEMP'].str[2]
df_presencial_cursos['CURSO'] = df_presencial_cursos['TEMP'].str[0]

#df_presencial_cursos.drop(columns=['TEMP'], inplace=True)
df_presencial_cursos.head()

,COD_CURSO,CURSO,SETOR_CURSO,TEMP,MODALIDADE,LOCAL
0,40001016027G0,ENGENHARIA AMBIENTAL E SANITÁRIA,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",Presencial,Pontal do Paraná
1,40001016047G0,FILOSOFIA,SETOR DE CIÊNCIAS HUMANAS,"[FILOSOFIA, Presencial, Curitiba]",Presencial,Curitiba
2,40001016030G0,ENGENHARIA CIVIL,SETOR DE TECNOLOGIA,"[ENGENHARIA CIVIL, Presencial, Curitiba]",Presencial,Curitiba
3,40001016005G0,ARQUITETURA E URBANISMO,SETOR DE TECNOLOGIA,"[ARQUITETURA E URBANISMO, Presencial, Curitiba]",Presencial,Curitiba
4,40001016076G0,MEDICINA VETERINÁRIA,SETOR DE CIÊNCIAS AGRÁRIAS,"[MEDICINA VETERINÁRIA, Presencial, Curitiba]",Presencial,Curitiba


In [5]:
columns = ['MODALIDADE','LOCAL']
for i in columns:
    dist_variavel(df_presencial_cursos,i,title=f'cursos/Distribuicao_de_{i}_before')

In [6]:
counts = df_presencial_cursos['MODALIDADE'].value_counts()
low_modalidades = counts[counts < 10].index
indices = df_presencial_cursos[df_presencial_cursos['MODALIDADE'].isin(low_modalidades)].index.tolist()
print('Indíces a serem arrumados: ',indices)

df_temp = df_presencial_cursos.loc[indices,:]

df_temp_expanded = pd.DataFrame(df_temp['TEMP'].tolist(), index=df_temp.index)
df_temp_expanded.columns = ['CURSO', 'MODALIDADE', 'LOCAL']
display(df_temp_expanded)

Indíces a serem arrumados:  [37, 67]


,CURSO,MODALIDADE,LOCAL
37,EDUCAÇÃO ESPECIAL E INCLUSIVA,a Distância,Curitiba
67,GESTÃO CULTURAL,a Distância,Curitiba


In [7]:
df_presencial_cursos = df_presencial_cursos.drop(indices).reset_index(drop=True)
df_presencial_cursos = df_presencial_cursos.drop(index=df_presencial_cursos.shape[0]-1)
df_presencial_cursos.head()

,COD_CURSO,CURSO,SETOR_CURSO,TEMP,MODALIDADE,LOCAL
0,40001016027G0,ENGENHARIA AMBIENTAL E SANITÁRIA,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",Presencial,Pontal do Paraná
1,40001016047G0,FILOSOFIA,SETOR DE CIÊNCIAS HUMANAS,"[FILOSOFIA, Presencial, Curitiba]",Presencial,Curitiba
2,40001016030G0,ENGENHARIA CIVIL,SETOR DE TECNOLOGIA,"[ENGENHARIA CIVIL, Presencial, Curitiba]",Presencial,Curitiba
3,40001016005G0,ARQUITETURA E URBANISMO,SETOR DE TECNOLOGIA,"[ARQUITETURA E URBANISMO, Presencial, Curitiba]",Presencial,Curitiba
4,40001016076G0,MEDICINA VETERINÁRIA,SETOR DE CIÊNCIAS AGRÁRIAS,"[MEDICINA VETERINÁRIA, Presencial, Curitiba]",Presencial,Curitiba


### 2.2 `df_presencial_disciplinas`

In [8]:
df_presencial_disciplinas['TEMP'] = df_presencial_disciplinas['CURSO'].str.split('-').apply(lambda x: [item.strip() for item in x])
df_presencial_disciplinas.head(2)

,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,CURSO,IDPROGRAMA,SETOR_CURSO,TEMP
0,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,40001016027G0,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,..."
1,EAS172-EAD,QUALIDADE DA ÁGUA,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,40001016027G0,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,..."


In [9]:


df_presencial_disciplinas['MODALIDADE'] = df_presencial_disciplinas['TEMP'].str[1]
df_presencial_disciplinas['LOCAL'] = df_presencial_disciplinas['TEMP'].str[2]
df_presencial_disciplinas['CURSO'] = df_presencial_disciplinas['TEMP'].str[0]

df_presencial_disciplinas

columns = ['MODALIDADE','LOCAL']
for i in columns:
    dist_variavel(df_presencial_disciplinas,i,title=f'disciplinas/Distribuicao_de_{i}_before')

In [10]:
counts = df_presencial_disciplinas['MODALIDADE'].value_counts()
low_modalidades = counts[counts < 10].index
indices_disciplinas = df_presencial_disciplinas[df_presencial_disciplinas['MODALIDADE'].isin(low_modalidades)].index.tolist()
print('Indíces a serem arrumados: ',indices_disciplinas)

df_temp = df_presencial_disciplinas.loc[indices_disciplinas,:]

df_temp_expanded = pd.DataFrame(df_temp['TEMP'].tolist(), index=df_temp.index)
display(df_temp_expanded)

Indíces a serem arrumados:  [960, 961, 962, 963, 964]


,0,1,2
960,GESTÃO CULTURAL,a Distância,Curitiba
961,GESTÃO CULTURAL,a Distância,Curitiba
962,GESTÃO CULTURAL,a Distância,Curitiba
963,GESTÃO CULTURAL,a Distância,Curitiba
964,GESTÃO CULTURAL,a Distância,Curitiba


In [11]:
print('Before: ',df_presencial_disciplinas.shape)
df_presencial_disciplinas = df_presencial_disciplinas.drop(indices_disciplinas).reset_index(drop=True)
print('After: ',df_presencial_disciplinas.shape)

Before:  (3897, 10)
After:  (3892, 10)


In [12]:
# Voltando a tabela ao padrão

df_presencial_disciplinas['CURSO'] = df_presencial_disciplinas['TEMP']
if all(col in df_presencial_disciplinas.columns for col in ['MODALIDADE', 'LOCAL','TEMP']):
    df_presencial_disciplinas = df_presencial_disciplinas.drop(columns=['MODALIDADE','LOCAL','TEMP'],axis=1)

df_presencial_disciplinas.head()

,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,CURSO,IDPROGRAMA,SETOR_CURSO
0,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",40001016027G0,CENTRO DE ESTUDOS DO MAR
1,EAS172-EAD,QUALIDADE DA ÁGUA,40001016027G0,Não,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",40001016027G0,CENTRO DE ESTUDOS DO MAR
2,EAS174,GEOPROCESSAMENTO,40001016027G0,Não,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",40001016027G0,CENTRO DE ESTUDOS DO MAR
3,PP015,CÁLCULO NUMÉRICO,40001016027G0,Não,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",40001016027G0,CENTRO DE ESTUDOS DO MAR
4,PP035-EAD,BIOQUÍMICA,40001016027G0,Não,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",40001016027G0,CENTRO DE ESTUDOS DO MAR


In [13]:

duplicados = df_presencial_disciplinas['COD_DISCIPLINA'].value_counts()

print("Top disciplinas duplicadas:")
print(duplicados[duplicados > 1])

# Vamos olhar um exemplo prático para entender O QUE está mudando
codigo_exemplo = duplicados[duplicados > 1].index[0]
print(f"\nExemplo de linhas para o código {codigo_exemplo}:")

# Exemplo 
# display(df_presencial_disciplinas[df_presencial_disciplinas['COD_DISCIPLINA'] == codigo_exemplo])

Top disciplinas duplicadas:
COD_DISCIPLINA
CE009     21
LIB038    17
CM303     12
CM310     11
SL52      10
          ..
CMM011     2
DEC012     2
AF068      2
ET200      2
DEE253     2
Name: count, Length: 290, dtype: int64

Exemplo de linhas para o código CE009:


### 2.3 `df_info_questionario`

Limpeza manual pela pequena quantidade de dados.

### 2.4 `df_info_perguntas`

Informações das possíveis perguntas: Fazer merge com outras tabelas.

### 2.5 `df_info_dimensoes`

Limpeza manual pela pequena quantidade de dados.

### 2.6 `df_presencial_pesquisa`

Esse é o nosso DataFrame principal, onde trabalharemos com os merges para o processamento final

In [14]:
cols_pesquisa = df_presencial_pesquisa.columns
cols_disciplinas = df_presencial_disciplinas.columns

common_cols = cols_disciplinas.difference(cols_pesquisa)
print(common_cols)

Index(['IDPROGRAMA'], dtype='object')


In [15]:
print(f'Shape before 1° merge: {df_presencial_pesquisa.shape}')
df_presencial_pesquisa = df_presencial_pesquisa.drop_duplicates()
print(f'Shape after drop_duplicates: {df_presencial_pesquisa.shape}')
# Disciplinas
df_presencial_pesquisa = pd.merge(
    left=df_presencial_pesquisa,
    right=df_presencial_disciplinas[['NOME_DISCIPLINA','COD_CURSO','IDPROGRAMA']].rename(columns={}),
    on=['NOME_DISCIPLINA','COD_CURSO'],
    how='left'
)
print(f'Shape after 1° merge: {df_presencial_pesquisa.shape}')
df_presencial_pesquisa = df_presencial_pesquisa.drop_duplicates()
print(f'Shape after drop_duplicates: {df_presencial_pesquisa.shape}')
df_presencial_pesquisa.head()

Shape before 1° merge: (215481, 15)
Shape after drop_duplicates: (215356, 15)
Shape after 1° merge: (229777, 16)
Shape after drop_duplicates: (215356, 16)


,ID_PESQUISA,ID_QUESTIONARIO,QUESTIONARIO,ID_PERGUNTA,PERGUNTA,RESPOSTA,SITUACAO,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,CURSO,SETOR_CURSO,DEPARTAMENTO,CODPROF,IDPROGRAMA
0,38663,523,Disciplina: avaliação da percepção discente so...,1732,"O plano de ensino da disciplina (objetivos, co...",Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259,40001016027G0
1,38663,523,Disciplina: avaliação da percepção discente so...,1733,"O plano da disciplina foi cumprido e, quando n...",Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259,40001016027G0
2,38663,523,Disciplina: avaliação da percepção discente so...,1734,A carga horária proposta para esta disciplina ...,Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259,40001016027G0
3,38663,523,Disciplina: avaliação da percepção discente so...,1735,"O conteúdo abordado é atual, coerente e articu...",Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259,40001016027G0
4,38663,523,Disciplina: avaliação da percepção discente so...,1736,As metodologias de ensino corresponderam aos p...,Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259,40001016027G0


## 3. Finalizando os Merges

In [16]:
df_merged_perguntas_dimensoes = pd.merge(
    left=df_info_perguntas.rename(columns={'Tipo_Pergunta':'Tipo_Perg'}),
    right=df_info_dimensoes[['Tipo_Perg','EIXO','DIMENSAO']],
    on='Tipo_Perg',
    how='left')

In [17]:
print(df_merged_perguntas_dimensoes.columns.difference(df_presencial_pesquisa.columns))

Index(['CL_PERGUNTA', 'DIMENSAO', 'EIXO', 'Ordem', 'Tipo_Perg'], dtype='object')


In [ ]:
print('Shape Before: ', df_presencial_pesquisa.shape)
df_presencial_pesquisa = pd.merge(
    left=df_presencial_pesquisa,
    right=df_merged_perguntas_dimensoes[['ID_PERGUNTA','CL_PERGUNTA', 'DIMENSAO', 'EIXO', 'Ordem', 'Tipo_Perg']],
    on='ID_PERGUNTA',
    how='left')
print('Shape After: ', df_presencial_pesquisa.shape)

Shape Before:  (215356, 16)
Shape After:  (215356, 21)


## 4. Identificando possíveis problemas

In [ ]:
sorted(df_presencial_pesquisa[['Ordem','ID_QUESTIONARIO','ID_PERGUNTA']].drop_duplicates().values.tolist(),key=lambda x: x[0]) # De todas as possíveis perguntas, somente aquelas referentes a disciplinas estão aqui. Para mais detalhes, veja: data\raw\Presencial2025_dados_raw\info_perguntas_presencial.csv

[[1, 523, 1732],
 [2, 523, 1733],
 [3, 523, 1734],
 [4, 523, 1735],
 [5, 523, 1736],
 [6, 523, 1762],
 [7, 523, 1737],
 [8, 523, 1738],
 [9, 523, 1739],
 [10, 523, 1740]]

In [20]:
df_presencial_pesquisa['DIMENSAO'].value_counts()

DIMENSAO
8.0    215356
Name: count, dtype: int64

# Testes

In [21]:
df_presencial_pesquisa.groupby('ID_PESQUISA')['RESPOSTA'].count()

ID_PESQUISA
38663     53
38664     36
38665     80
38666     60
38667     50
        ... 
46944     19
46947    144
46948    181
46949    252
46951    180
Name: RESPOSTA, Length: 3729, dtype: int64

In [22]:
df_presencial_pesquisa['RESPOSTA'].unique()
df_presencial_pesquisa['VALOR_RESPOSTA'] = pd.Series([1 if i in ['Sim','Concordo'] else 0 if i == 'Desconheço' else -1 for i in df_presencial_pesquisa['RESPOSTA']])
df_presencial_pesquisa.head(1)

,ID_PESQUISA,ID_QUESTIONARIO,QUESTIONARIO,ID_PERGUNTA,PERGUNTA,RESPOSTA,SITUACAO,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,...,SETOR_CURSO,DEPARTAMENTO,CODPROF,IDPROGRAMA,CL_PERGUNTA,DIMENSAO,EIXO,Ordem,Tipo_Perg,VALOR_RESPOSTA
0,38663,523,Disciplina: avaliação da percepção discente so...,1732,"O plano de ensino da disciplina (objetivos, co...",Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259,40001016027G0,Disciplina,8.0,1,1,AvPDSPF,-1


In [23]:
temp = df_presencial_pesquisa.groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

In [24]:
mask1 = df_presencial_pesquisa['VALOR_RESPOSTA'] > 0
temp1 = df_presencial_pesquisa[mask1].groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

mask2 = df_presencial_pesquisa['VALOR_RESPOSTA'] == 0
temp2 = df_presencial_pesquisa[mask2].groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

mask3 = df_presencial_pesquisa['VALOR_RESPOSTA'] < 0
temp3 = df_presencial_pesquisa[mask3].groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

temp_final = pd.concat([temp,temp1, temp2, temp3], axis=1, keys=['Total','Positivas', 'Desconheço', 'Negativas'])
temp_final.corr(method='spearman')

,Total,Positivas,Desconheço,Negativas
Total,1.000000,0.935031,0.316420,0.644008
Positivas,0.935031,1.000000,0.119928,0.440809
Desconheço,0.316420,0.119928,1.000000,0.149741
Negativas,0.644008,0.440809,0.149741,1.000000


In [25]:
len(df_presencial_pesquisa['ID_PESQUISA'].unique().tolist())

3729